#Data Mining: Lab3

## Implementation of Autoencoder
---

Get MNIST training set and data set and do some preprocessing.

In [1]:
from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.

x_test  = x_test .astype('float32') / 255.

x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))

x_test  = np.reshape(x_test,  (len(x_test),  28, 28, 1))

---

Create autoencoder model with encoder and decoder.
Ecnoder has 1 input, 3 hidden and 1 output layers.
Decoder has 1 input, 3 hidden and 1 output layers too.
With this hyperparameters we get best answer, so we deside to stop on this variant.
Layers: 784 -> 500 -> 500 -> 2000 -> 2000 -> 500 -> 500 -> 784

In [2]:
from keras import Sequential
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model

def create_deep_dense_ae():
    # Final dimension
    encoding_dim = 10

    # Encoder
    input_img = Input(shape=(28, 28, 1))
    flat_img = Flatten()(input_img)
    x = Dense(500, activation='relu')(flat_img)
    x = Dense(500, activation='relu')(x)
    x = Dense(2000, activation='relu')(x)
    encoded = Dense(encoding_dim, activation='linear')(x)

    # Decoder
    input_encoded = Input(shape=(encoding_dim,))
    x = Dense(2000, activation='relu')(input_encoded)
    x = Dense(500, activation='relu')(x)
    x = Dense(500, activation='relu')(x)
    flat_decoded = Dense(28 * 28, activation='sigmoid')(x)
    decoded = Reshape((28, 28, 1))(flat_decoded)

    # Models
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

Using TensorFlow backend.


---

Fit model and save modeles.


In [3]:
encoder, decoder, autoencoder = create_deep_dense_ae()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()
autoencoder.fit(x_train, x_train,
                epochs=40,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

encoder.save("encoder_model_saved.h5")
decoder.save("decoder_model_saved.h5")